# Acquire

In [2]:
import env
import pandas as pd
import prepare_utils as prep

import os

In [3]:
import seaborn as sns
import re


Read tables

In [4]:
db = 'superstore_db'

categories_query = '''
                    SELECT *
                    FROM categories;
                    '''

customers_query = '''
                    SELECT *
                    FROM customers;
                    '''

orders_query = '''
                SELECT *
                FROM orders;
                '''

products_query = '''
                    SELECT *
                    FROM products;
                    '''

regions_query = '''
                    SELECT *
                    FROM regions;
                    '''




categories = pd.read_sql(categories_query, 
                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')

customers = pd.read_sql(customers_query, 
                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')

orders = pd.read_sql(orders_query, 
                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')

products = pd.read_sql(products_query, 
                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')

regions = pd.read_sql(regions_query, 
                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')

Join tables

In [5]:
df = orders.merge(customers, on='Customer ID', how='left')
df = df.merge(categories, on='Category ID', how='left')
df = df.merge(regions, on='Region ID', how='left')
df = df.merge(products, on='Product ID', how='left')
df

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Segment,Country,City,State,Postal Code,...,Quantity,Discount,Profit,Category ID,Region ID,Customer Name,Category,Sub-Category,Region Name,Product Name
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,...,2.0,0.00,41.9136,1,1,Claire Gute,Furniture,Bookcases,South,Bush Somerset Collection Bookcase
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,...,3.0,0.00,219.5820,2,1,Claire Gute,Furniture,Chairs,South,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036.0,...,2.0,0.00,6.8714,3,2,Darrin Van Huff,Office Supplies,Labels,West,Self-Adhesive Address Labels for Typewriters b...
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,5.0,0.45,-383.0310,4,1,Sean O'Donnell,Furniture,Tables,South,Bretford CR4500 Series Slim Rectangular Table
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,2.0,0.20,2.5164,5,1,Sean O'Donnell,Office Supplies,Storage,South,Eldon Fold 'N Roll Cart System
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,CA-2016-106894,2016-02-07,2016-02-07,Same Day,CA-12265,Consumer,United States,Springfield,Virginia,22153.0,...,3.0,0.00,3.6018,3,1,Christina Anderson,Office Supplies,Labels,South,Permanent Self-Adhesive File Folder Labels for...
1730,CA-2014-125136,2014-09-28,2014-10-03,Standard Class,KD-16495,Corporate,United States,Fayetteville,North Carolina,28314.0,...,8.0,0.20,31.2832,11,1,Keith Dawkins,Office Supplies,Paper,South,White GlueTop Scratch Pads
1731,CA-2014-125136,2014-09-28,2014-10-03,Standard Class,KD-16495,Corporate,United States,Fayetteville,North Carolina,28314.0,...,2.0,0.20,3.7408,11,1,Keith Dawkins,Office Supplies,Paper,South,Xerox 195
1732,US-2016-131149,2016-07-10,2016-07-14,Standard Class,LH-17155,Consumer,United States,Dallas,Texas,75081.0,...,3.0,0.20,-33.8040,5,3,Logan Haushalter,Office Supplies,Storage,Central,"Fellowes Strictly Business Drawer File, Letter..."


In [6]:
def acquire_store_data():

    filename = 'data/store_data_raw'
    
    # if file doesn't exist, acquire and cache
    if not os.path.isfile(filename):
    
        db = 'superstore_db'
        
        categories_query = "SELECT * FROM categories;"
        customers_query  = "SELECT * FROM customers;"
        orders_query     = "SELECT * FROM orders;"
        products_query   = "SELECT * FROM products;"
        regions_query    = "SELECT * FROM regions;"

        
        categories = pd.read_sql(categories_query, 
                                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')
        customers  = pd.read_sql(customers_query, 
                                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')
        orders     = pd.read_sql(orders_query, 
                                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')
        products   = pd.read_sql(products_query, 
                                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')
        regions    = pd.read_sql(regions_query, 
                                 f'mysql+pymysql://{env.user}:{env.pwd}@{env.host}/{db}')
        
        # join tables
        df = orders.merge(customers, on='Customer ID', how='left')
        df = df.merge(categories, on='Category ID', how='left')
        df = df.merge(regions, on='Region ID', how='left')
        df = df.merge(products, on='Product ID', how='left')
        df

        df.to_csv(filename, index=False)
        
        return df
    
    return pd.read_csv(filename)

In [7]:
df = acquire_store_data()

In [8]:
df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Segment,Country,City,State,Postal Code,...,Quantity,Discount,Profit,Category ID,Region ID,Customer Name,Category,Sub-Category,Region Name,Product Name
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,...,2.0,0.00,41.9136,1,1,Claire Gute,Furniture,Bookcases,South,Bush Somerset Collection Bookcase
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,...,3.0,0.00,219.5820,2,1,Claire Gute,Furniture,Chairs,South,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036.0,...,2.0,0.00,6.8714,3,2,Darrin Van Huff,Office Supplies,Labels,West,Self-Adhesive Address Labels for Typewriters b...
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,5.0,0.45,-383.0310,4,1,Sean O'Donnell,Furniture,Tables,South,Bretford CR4500 Series Slim Rectangular Table
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,2.0,0.20,2.5164,5,1,Sean O'Donnell,Office Supplies,Storage,South,Eldon Fold 'N Roll Cart System


# Prepare


In [9]:
for col in df.columns:
    display(df[col].value_counts())

US-2016-108504    11
CA-2015-131338    10
CA-2017-117457     9
US-2016-110156     8
US-2017-118087     8
                  ..
CA-2016-152632     1
CA-2016-167584     1
CA-2014-134677     1
CA-2016-161781     1
CA-2017-150707     1
Name: Order ID, Length: 841, dtype: int64

2015-09-26    14
2016-10-28    14
2017-12-08    13
2016-06-20    13
2016-11-24    13
              ..
2014-03-10     1
2014-04-21     1
2015-10-22     1
2015-08-22     1
2015-02-08     1
Name: Order Date, Length: 570, dtype: int64

2016-11-24    17
2016-11-26    13
2017-11-21    11
2017-12-12    11
2016-02-05    11
              ..
2014-10-10     1
2015-05-14     1
2016-07-19     1
2016-07-07     1
2017-09-22     1
Name: Ship Date, Length: 608, dtype: int64

Standard Class    980
Second Class      354
First Class       326
Same Day           74
Name: Ship Mode, dtype: int64

ZC-21910    16
NP-18325    15
JE-15745    15
BM-11140    13
RB-19795    13
            ..
RO-19780     1
SS-20590     1
LH-16750     1
XP-21865     1
FM-14215     1
Name: Customer ID, Length: 525, dtype: int64

...

Zuschuss Carroll    16
Naresj Patel        15
Joel Eaton          15
Becky Martin        13
Ross Baird          13
                    ..
Rose O'Brian         1
Sonia Sunley         1
Larry Hughes         1
Xylona Preis         1
Filia McAdams        1
Name: Customer Name, Length: 525, dtype: int64

Office Supplies    1055
Furniture           360
Technology          319
Name: Category, dtype: int64

Binders        263
Paper          224
Furnishings    168
Storage        162
Phones         157
Art            149
Accessories    131
Chairs          95
Appliances      71
Labels          67
Tables          62
Envelopes       48
Fasteners       43
Bookcases       35
Supplies        28
Machines        21
Copiers         10
Name: Sub-Category, dtype: int64

East       556
West       541
Central    389
South      248
Name: Region Name, dtype: int64

Easy-staple paper                                                 10
Staples                                                            9
Xerox 1881                                                         7
Staple envelope                                                    6
DAX Value U-Channel Document Frames, Easel Back                    6
                                                                  ..
Jabra SPEAK 410                                                    1
Verbatim 25 GB 6x Blu-ray Single Layer Recordable Disc, 1/Pack     1
Office Star - Mesh Screen back chair with Vinyl seat               1
Premium Transparent Presentation Covers by GBC                     1
White GlueTop Scratch Pads                                         1
Name: Product Name, Length: 1091, dtype: int64

- rename columns

In [10]:
df.columns = [re.sub(r'[^a-zA-Z]', '_', col.lower()) for col in df.columns]
df.columns

Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id',
       'segment', 'country', 'city', 'state', 'postal_code', 'product_id',
       'sales', 'quantity', 'discount', 'profit', 'category_id', 'region_id',
       'customer_name', 'category', 'sub_category', 'region_name',
       'product_name'],
      dtype='object')

- check missing values

In [11]:
df.isna().sum()

order_id         0
order_date       0
ship_date        0
ship_mode        0
customer_id      0
segment          0
country          0
city             0
state            0
postal_code      0
product_id       0
sales            0
quantity         0
discount         0
profit           0
category_id      0
region_id        0
customer_name    0
category         0
sub_category     0
region_name      0
product_name     0
dtype: int64

In [12]:
prep.identify_cols_with_white_space(df)

[]

- drop foreign key columns

In [13]:
df = df.drop(columns=['customer_id','product_id',
                      'category_id', 'region_id'])

- check data types

In [14]:
df.head()

,order_id,order_date,ship_date,ship_mode,segment,country,city,state,postal_code,sales,quantity,discount,profit,customer_name,category,sub_category,region_name,product_name
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,Consumer,United States,Henderson,Kentucky,42420.0,261.9600,2.0,0.00,41.9136,Claire Gute,Furniture,Bookcases,South,Bush Somerset Collection Bookcase
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,Consumer,United States,Henderson,Kentucky,42420.0,731.9400,3.0,0.00,219.5820,Claire Gute,Furniture,Chairs,South,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,Corporate,United States,Los Angeles,California,90036.0,14.6200,2.0,0.00,6.8714,Darrin Van Huff,Office Supplies,Labels,West,Self-Adhesive Address Labels for Typewriters b...
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311.0,957.5775,5.0,0.45,-383.0310,Sean O'Donnell,Furniture,Tables,South,Bretford CR4500 Series Slim Rectangular Table
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311.0,22.3680,2.0,0.20,2.5164,Sean O'Donnell,Office Supplies,Storage,South,Eldon Fold 'N Roll Cart System


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734 entries, 0 to 1733
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       1734 non-null   object 
 1   order_date     1734 non-null   object 
 2   ship_date      1734 non-null   object 
 3   ship_mode      1734 non-null   object 
 4   segment        1734 non-null   object 
 5   country        1734 non-null   object 
 6   city           1734 non-null   object 
 7   state          1734 non-null   object 
 8   postal_code    1734 non-null   float64
 9   sales          1734 non-null   float64
 10  quantity       1734 non-null   float64
 11  discount       1734 non-null   float64
 12  profit         1734 non-null   float64
 13  customer_name  1734 non-null   object 
 14  category       1734 non-null   object 
 15  sub_category   1734 non-null   object 
 16  region_name    1734 non-null   object 
 17  product_name   1734 non-null   object 
dtypes: float

In [16]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['ship_date'] = pd.to_datetime(df['ship_date'])

In [17]:
df['quantity'] = df['quantity'].astype(int)

Feature Engineering

In [18]:
df['order_month'] = df['order_date'].dt.month
df['order_year'] = df['order_date'].dt.year


In [19]:
df['country'].value_counts()

United States    1734
Name: country, dtype: int64

In [20]:
df

,order_id,order_date,ship_date,ship_mode,segment,country,city,state,postal_code,sales,quantity,discount,profit,customer_name,category,sub_category,region_name,product_name,order_month,order_year
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,Consumer,United States,Henderson,Kentucky,42420.0,261.9600,2,0.00,41.9136,Claire Gute,Furniture,Bookcases,South,Bush Somerset Collection Bookcase,11,2016
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,Consumer,United States,Henderson,Kentucky,42420.0,731.9400,3,0.00,219.5820,Claire Gute,Furniture,Chairs,South,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",11,2016
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,Corporate,United States,Los Angeles,California,90036.0,14.6200,2,0.00,6.8714,Darrin Van Huff,Office Supplies,Labels,West,Self-Adhesive Address Labels for Typewriters b...,6,2016
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311.0,957.5775,5,0.45,-383.0310,Sean O'Donnell,Furniture,Tables,South,Bretford CR4500 Series Slim Rectangular Table,10,2015
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311.0,22.3680,2,0.20,2.5164,Sean O'Donnell,Office Supplies,Storage,South,Eldon Fold 'N Roll Cart System,10,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,CA-2016-106894,2016-02-07,2016-02-07,Same Day,Consumer,United States,Springfield,Virginia,22153.0,7.8300,3,0.00,3.6018,Christina Anderson,Office Supplies,Labels,South,Permanent Self-Adhesive File Folder Labels for...,2,2016
1730,CA-2014-125136,2014-09-28,2014-10-03,Standard Class,Corporate,United States,Fayetteville,North Carolina,28314.0,96.2560,8,0.20,31.2832,Keith Dawkins,Office Supplies,Paper,South,White GlueTop Scratch Pads,9,2014
1731,CA-2014-125136,2014-09-28,2014-10-03,Standard Class,Corporate,United States,Fayetteville,North Carolina,28314.0,10.6880,2,0.20,3.7408,Keith Dawkins,Office Supplies,Paper,South,Xerox 195,9,2014
1732,US-2016-131149,2016-07-10,2016-07-14,Standard Class,Consumer,United States,Dallas,Texas,75081.0,338.0400,3,0.20,-33.8040,Logan Haushalter,Office Supplies,Storage,Central,"Fellowes Strictly Business Drawer File, Letter...",7,2016


- Univariate analysis
    - outliers
    - feature distributions
    - target distributions

In [22]:
df.groupby('discount')['profit'].mean()

discount
0.00     63.294775
0.10    160.891143
0.15      5.084475
0.20     20.978555
0.30    -59.368884
0.32   -102.990100
0.40   -144.433722
0.45   -211.530540
0.50   -506.090250
0.60    -59.002942
0.70    -69.546530
0.80    -91.695554
Name: profit, dtype: float64